In [5]:
# Install necessary libraries
%pip install fastapi uvicorn python-multipart pillow tensorflow pyngrok numpy torch 

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
from PIL import Image
import io
import uvicorn
import asyncio
import torch
import torchvision.transforms as transforms
import os

ModuleNotFoundError: No module named 'torch'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to the saved model
MODEL_PATH = "/content/drive/MyDrive/FYP-2/Model_prep/Model/truth_lens_xception_2.h5"
print("Model exists:", os.path.exists(MODEL_PATH))

In [ ]:
loaded_model = load_model(MODEL_PATH)

In [ ]:
# Initialize FastAPI application
app = FastAPI()

In [ ]:
try:
    model = load_model(MODEL_PATH)
    model.trainable = False
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

In [ ]:
# Preprocessing function
def preprocess_image(image):
    """
    Preprocesses the uploaded image for the Xception model.
    - Converts image to RGB.
    - Resizes and normalizes the image.
    """
    image = image.convert("RGB")  # Ensure RGB format
    image = image.resize((224, 224))  # Resize to model's required size
    image = np.array(image)  # Convert to numpy array
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension (1, 224, 224, 3)
    return image

In [ ]:
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        # Validate file type
        if file.content_type not in ["image/jpeg", "image/png"]:
            raise HTTPException(status_code=400, detail="Unsupported file type. Please upload a JPG or PNG image.")

        # Read and process image
        image_bytes = await file.read()
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        # Preprocess image
        processed_image = preprocess_image(image)

        # Make prediction
        predictions = model.predict(processed_image)
        confidence = float(predictions.max()) * 100
        predicted_class = int(predictions.argmax())

        # Adjust classification logic based on confidence
        prediction_label = "Real" if confidence >= 90 else "Fake"

        return JSONResponse(content={
            "prediction": prediction_label,
            "confidence": f"{confidence:.2f}%"
        })

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error processing image: {e}")

In [ ]:
# Health check endpoint
@app.get("/")
async def root():
    """
    Health check endpoint to verify the server is running.
    """
    return {"message": "Truth Lens backend is running!"}


In [ ]:
# To run FastAPI in Colab
def run_app():
    """
    Runs the FastAPI server using Uvicorn.
    """
    config = uvicorn.Config(app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config)

    loop = asyncio.get_event_loop() # Get the current running event loop.
    loop.run_until_complete(server.serve()) # This code runs an asynchronous task, so you call it through loop.run_until_complete

    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
from pyngrok import ngrok

# Set the Ngrok authentication token
ngrok.set_auth_token("2s4wxdhWi2WjFU2kylwee8qHqbH_7XSvxiPspoJQshE3tFA17")

# Expose the local server to the internet
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

In [ ]:
import nest_asyncio
import uvicorn

@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI with Ngrok on Colab!"}

# Patch the event loop
nest_asyncio.apply()

# Set the Ngrok authentication token
ngrok.set_auth_token("2s4wxdhWi2WjFU2kylwee8qHqbH_7XSvxiPspoJQshE3tFA17")  # Replace with your Ngrok auth token

# Expose the local server to the internet
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

#*---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------*